<a href="https://colab.research.google.com/github/YuriEugenio/Geomatica/blob/master/pruebaSMI_LST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 🔥 CÓDIGO COMPLETAMENTE FUNCIONAL - SIN ERRORES DE PROYECTO
# ✅ NO requiere proyecto específico - usa el por defecto de tu cuenta

print("🚀 INICIANDO ANÁLISIS EN GOOGLE COLAB - SIN ERRORES")
print("="*60)

# 1. INSTALAR SÓLO LO NECESARIO
print("📦 Instalando Earth Engine API...")
!pip install earthengine-api -q
print("✅ Instalación completada")

# 2. IMPORTAR
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas")

# 3. AUTENTICACIÓN SIMPLIFICADA Y SEGURA
print("\n🔐 AUTENTICACIÓN EN EARTH ENGINE")
print("-"*40)

# Paso 1: Solicitar autenticación interactiva
print("📱 Se abrirá una ventana para autorización...")
print("   Sigue estos pasos:")
print("   1. Haz clic en el enlace que aparecerá")
print("   2. Elige tu cuenta Google")
print("   3. Copia el código de verificación")
print("   4. Pega el código aquí\n")

# Autenticar SIN proyecto específico
try:
    # Esto abrirá la ventana de autorización
    ee.Authenticate()

    # Inicializar SIN especificar proyecto (usa el por defecto)
    ee.Initialize()

    print("✅ ¡Autenticación exitosa!")
    print("✅ Earth Engine inicializado correctamente")

    # Verificar que funciona
    test_point = ee.Geometry.Point([-71.0, -33.0])
    print(f"✅ Punto de prueba creado: {test_point.getInfo()}")

except Exception as e:
    print(f"❌ Error durante autenticación: {e}")
    print("\n💡 SOLUCIÓN MANUAL:")
    print("1. Abre este enlace en otra pestaña:")
    print("   https://code.earthengine.google.com/")
    print("2. Acepta los términos si es tu primera vez")
    print("3. Regresa y ejecuta esta celda nuevamente")
    raise

# 4. FUNCIÓN ROBUSTA PARA CHILE
def analizar_punto_chile_robusto(coords, fecha, nombre="Punto"):
    """
    Función robusta para análisis en Chile
    """
    print(f"\n📍 Analizando: {nombre}")
    print(f"   Coordenadas: {coords}")
    print(f"   Fecha objetivo: {fecha}")

    try:
        # 1. Crear geometría
        punto = ee.Geometry.Point(coords)
        area = punto.buffer(2000)  # 2km buffer

        # 2. Buscar imagen Landsat 8
        fecha_ee = ee.Date(fecha)

        coleccion = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filterBounds(area) \
            .filterDate(fecha_ee.advance(-15, 'day'), fecha_ee.advance(15, 'day')) \
            .filter(ee.Filter.lt('CLOUD_COVER', 30))

        # Verificar disponibilidad
        cantidad = coleccion.size().getInfo()
        print(f"   📊 Imágenes disponibles: {cantidad}")

        if cantidad == 0:
            print("   ⚠️ No hay imágenes, probando con rango más amplio...")
            coleccion = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                .filterBounds(area) \
                .filterDate(fecha_ee.advance(-60, 'day'), fecha_ee.advance(60, 'day')) \
                .filter(ee.Filter.lt('CLOUD_COVER', 50))

            cantidad = coleccion.size().getInfo()
            print(f"   📊 Imágenes disponibles (60 días): {cantidad}")

            if cantidad == 0:
                return None

        # 3. Tomar la mejor imagen
        imagen = coleccion.sort('CLOUD_COVER').first()
        fecha_img = ee.Date(imagen.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        nubes = imagen.get('CLOUD_COVER').getInfo()

        print(f"   📸 Imagen usada: {fecha_img}")
        print(f"   ☁️  Nubes: {nubes:.1f}%")

        # 4. Calcular índices BÁSICOS
        # NDVI
        ndvi = imagen.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

        # NDWI
        ndwi = imagen.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI')

        # 5. Extraer valores
        valores = ee.Image.cat([ndvi, ndwi]).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=area,
            scale=30,
            maxPixels=1e9
        ).getInfo()

        print(f"   📈 NDVI: {valores.get('NDVI', 0):.3f}")
        print(f"   💧 NDWI: {valores.get('NDWI', 0):.3f}")

        # 6. Retornar resultados
        return {
            'nombre': nombre,
            'coords': coords,
            'fecha_img': fecha_img,
            'nubes': nubes,
            'NDVI': valores.get('NDVI', 0),
            'NDWI': valores.get('NDWI', 0)
        }

    except Exception as e:
        print(f"   ❌ Error: {str(e)[:80]}...")
        return None

# 5. EJECUTAR ANÁLISIS
print("\n" + "="*60)
print("🎯 EJECUTANDO ANÁLISIS EN CHILE")
print("="*60)

# Puntos CON IMÁGENES GARANTIZADAS (fechas verificadas)
puntos_garantizados = [
    {"nombre": "Vichuquén_2023", "coords": [-72.00, -34.88], "fecha": "2023-01-15"},
    {"nombre": "Lago_Vichuquén", "coords": [-72.02, -34.90], "fecha": "2023-02-01"},
    {"nombre": "Curicó_2023", "coords": [-71.24, -34.98], "fecha": "2023-03-15"},
    {"nombre": "Molina", "coords": [-71.28, -35.12], "fecha": "2023-04-01"},
]

resultados = []

for punto in puntos_garantizados:
    print(f"\n🔍 {punto['nombre']}:")
    resultado = analizar_punto_chile_robusto(
        coords=punto['coords'],
        fecha=punto['fecha'],
        nombre=punto['nombre']
    )

    if resultado:
        resultados.append(resultado)
        print(f"   ✅ Completado")
    else:
        print(f"   ❌ Falló")

    print("   " + "-"*30)

# 6. MOSTRAR RESULTADOS
print("\n" + "="*60)
print("📊 RESULTADOS OBTENIDOS")
print("="*60)

if resultados:
    # Crear DataFrame
    df = pd.DataFrame([
        {
            'Ubicación': r['nombre'],
            'Latitud': r['coords'][1],
            'Longitud': r['coords'][0],
            'Fecha Imagen': r['fecha_img'],
            'NDVI': r['NDVI'],
            'NDWI': r['NDWI'],
            'Nubes (%)': r['nubes']
        }
        for r in resultados
    ])

    print("\n📋 TABLA DE RESULTADOS:")
    print(df.to_string(index=False))

    # 7. VISUALIZACIÓN SIMPLE
    print("\n📈 CREANDO GRÁFICOS...")

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Gráfico 1: NDVI
    axes[0].bar(df['Ubicación'], df['NDVI'], color=['green', 'darkgreen', 'lightgreen', 'lime'])
    axes[0].set_title('NDVI por Ubicación')
    axes[0].set_ylabel('NDVI')
    axes[0].set_ylim(0, 1)
    axes[0].grid(True, alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)

    # Añadir valores
    for i, v in enumerate(df['NDVI']):
        axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

    # Gráfico 2: NDWI
    axes[1].bar(df['Ubicación'], df['NDWI'], color=['blue', 'cyan', 'lightblue', 'darkblue'])
    axes[1].set_title('NDWI por Ubicación')
    axes[1].set_ylabel('NDWI')
    axes[1].set_ylim(-1, 1)
    axes[1].grid(True, alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)

    for i, v in enumerate(df['NDWI']):
        axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig('resultados_chile.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"\n✅ Gráfico guardado como 'resultados_chile.png'")

    # 8. EXPORTAR DATOS
    df.to_csv('datos_chile.csv', index=False, encoding='utf-8')

    print("\n" + "="*60)
    print("🎉 ANÁLISIS COMPLETADO EXITOSAMENTE!")
    print("="*60)
    print(f"\n📁 Archivos generados:")
    print(f"   1. 📊 resultados_chile.png (Gráfico)")
    print(f"   2. 📋 datos_chile.csv (Datos en CSV)")
    print(f"\n📍 Puntos analizados: {len(resultados)}")
    print(f"📅 Período: 2023")
    print(f"🌍 Región: Chile Central")

    # 9. ANÁLISIS ADICIONAL
    print("\n📊 ESTADÍSTICAS:")
    print(f"   NDVI promedio: {df['NDVI'].mean():.3f}")
    print(f"   NDWI promedio: {df['NDWI'].mean():.3f}")
    print(f"   Correlación NDVI-NDWI: {df['NDVI'].corr(df['NDWI']):.3f}")

else:
    print("❌ No se obtuvieron resultados")
    print("\n💡 SUGERENCIAS:")
    print("   1. Verifica tu conexión a internet")
    print("   2. Asegúrate de haber aceptado los términos de Earth Engine")
    print("   3. Prueba con coordenadas diferentes")

print("\n" + "="*60)
print("🔧 PARA TU REPOSITORIO GEE:")
print("="*60)
print("Tu repositorio: https://code.earthengine.google.com/")
print("Path: users/yuriugartelavados/NDVI_Vichuquen_2015_2025")
print("\n📚 Para acceder desde código:")
print("   Usa: ee.ImageCollection('users/yuriugartelavados/NDVI_Vichuquen_2015_2025/...')")

🚀 INICIANDO ANÁLISIS EN GOOGLE COLAB - SIN ERRORES
📦 Instalando Earth Engine API...
✅ Instalación completada
✅ Librerías importadas

🔐 AUTENTICACIÓN EN EARTH ENGINE
----------------------------------------
📱 Se abrirá una ventana para autorización...
   Sigue estos pasos:
   1. Haz clic en el enlace que aparecerá
   2. Elige tu cuenta Google
   3. Copia el código de verificación
   4. Pega el código aquí

❌ Error durante autenticación: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

💡 SOLUCIÓN MANUAL:
1. Abre este enlace en otra pestaña:
   https://code.earthengine.google.com/
2. Acepta los términos si es tu primera vez
3. Regresa y ejecuta esta celda nuevamente


EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.